In [33]:
import numpy as np
import pandas as pd 
import allel
import zarr
import seaborn as sns

#### Fst/PBS analysis on RNA-Seq data - Freebayes ploidy 10

Here, I want to do a gene-based FST and gene-based PBS analysis, between treatment groups. Comparisons have been stored in the file data/DE.comparison.list.

In [5]:
comparisons = pd.read_csv("../../data/DE.comparison.list", header=None)
comparisons.columns = ['contrast']

In [11]:
comparisons.contrast.str.split("_", expand=True)

,0,1
0,ContAngola,FenAngola
1,NewOrleans,ContAngola
2,Rockefeller,ContAngola


In [12]:
vcf1 = allel.read_vcf("../../data/variants/variants.freebayes.1.vcf")

### Filtering the input VCFs for quality

There are some crazy high quality values so to get a view from the data (by histogram), I just need to remove some outliers first before plotting. Ill use the function `is_outlier()`, located in `sweep_tools.py`.

Lets plot this is a histogram and see the distribution of quality scores.

In [53]:
qual = vcf1['variants/QUAL']
fltqual = qual[~is_outlier(qual)]

sns.distplot(fltqual)

Lets remove variants with a QUAL score below 30. We also need to take care of missingness: we expect high rates here due to this being RNA-Seq data, and therefore some treatments may not express genes entirely, eliminating the possibility of finding SNPs.

In [73]:
passfilter = np.log

397287

In [79]:
vcf1['calldata/GT']

array([[[0, 0]],

       [[0, 0]],

       [[0, 0]],

       ...,

       [[1, 1]],

       [[1, 1]],

       [[1, 1]]], dtype=int8)

In [76]:
allel.GenotypeArray(vcf1['calldata/GT'])

,0
0,0/0
1,0/0
2,0/0
...,...
504332,1/1
504333,1/1
504334,1/1


In [63]:
(vcf1['variants/QUAL'] < 2).sum()

34392